In [1]:
import overpy

api = overpy.Overpass()


In [6]:
# fetch all ways and nodes
example_query = """
    way(50.746,7.154,50.748,7.157) ["highway"];
    (._;>;);
    out body;
    """

test_query = """<query type="way">
    <around lat="33.255202" lon="-116.258510" radius="1000"/>
</query>
<union>
  <item/>
  <recurse type="down"/>
</union>
<print/>"""

result = api.query(test_query)

for way in result.ways:
    print("Name: %s" % way.tags.get("name", "n/a"))
    print("  Highway: %s" % way.tags.get("highway", "n/a"))
    print("  Nodes:")
    for node in way.nodes:
        print("    Lat: %f, Lon: %f" % (node.lat, node.lon))

Name: Borrego Salton Seaway
  Highway: secondary
  Nodes:
    Lat: 33.279640, Lon: -116.083199
    Lat: 33.279682, Lon: -116.083633
    Lat: 33.279699, Lon: -116.084795
    Lat: 33.279705, Lon: -116.086052
    Lat: 33.280403, Lon: -116.090652
    Lat: 33.281002, Lon: -116.096224
    Lat: 33.281023, Lon: -116.096463
    Lat: 33.281272, Lon: -116.098622
    Lat: 33.281442, Lon: -116.099602
    Lat: 33.282085, Lon: -116.103302
    Lat: 33.282333, Lon: -116.104980
    Lat: 33.282719, Lon: -116.106956
    Lat: 33.283234, Lon: -116.108841
    Lat: 33.283390, Lon: -116.109596
    Lat: 33.283497, Lon: -116.110733
    Lat: 33.283501, Lon: -116.111298
    Lat: 33.283440, Lon: -116.111923
    Lat: 33.283306, Lon: -116.112602
    Lat: 33.282455, Lon: -116.115463
    Lat: 33.282337, Lon: -116.116226
    Lat: 33.282152, Lon: -116.120503
    Lat: 33.282021, Lon: -116.122072
    Lat: 33.281997, Lon: -116.123398
    Lat: 33.282124, Lon: -116.124390
    Lat: 33.282362, Lon: -116.125294
    Lat: 33.28370

    Lat: 33.225003, Lon: -116.275241
    Lat: 33.224722, Lon: -116.275188
    Lat: 33.224578, Lon: -116.275099
    Lat: 33.224344, Lon: -116.274876
    Lat: 33.224137, Lon: -116.274619
    Lat: 33.224108, Lon: -116.274336
    Lat: 33.224140, Lon: -116.274085
    Lat: 33.223882, Lon: -116.273703
    Lat: 33.223865, Lon: -116.273638
    Lat: 33.223591, Lon: -116.273000
    Lat: 33.223630, Lon: -116.272759
    Lat: 33.223765, Lon: -116.272191
    Lat: 33.223832, Lon: -116.271766
    Lat: 33.223740, Lon: -116.271453
    Lat: 33.223817, Lon: -116.270822
    Lat: 33.223747, Lon: -116.270497
    Lat: 33.223570, Lon: -116.270424
    Lat: 33.223285, Lon: -116.270186
    Lat: 33.223011, Lon: -116.269800
    Lat: 33.222948, Lon: -116.269428
    Lat: 33.222855, Lon: -116.269082
    Lat: 33.222995, Lon: -116.268739
    Lat: 33.223025, Lon: -116.268375
    Lat: 33.223132, Lon: -116.268023
    Lat: 33.223103, Lon: -116.267780
    Lat: 33.222877, Lon: -116.267550
    Lat: 33.222851, Lon: -116.267387
 

  Highway: n/a
  Nodes:
    Lat: 33.256608, Lon: -116.234209
    Lat: 33.256613, Lon: -116.234306
    Lat: 33.256630, Lon: -116.234340
    Lat: 33.256635, Lon: -116.234379
    Lat: 33.256661, Lon: -116.234448
    Lat: 33.256641, Lon: -116.234473
    Lat: 33.256639, Lon: -116.234495
    Lat: 33.256657, Lon: -116.234547
    Lat: 33.256639, Lon: -116.234619
    Lat: 33.256570, Lon: -116.234704
    Lat: 33.256535, Lon: -116.234784
    Lat: 33.256516, Lon: -116.234913
    Lat: 33.256500, Lon: -116.234973
    Lat: 33.256443, Lon: -116.235011
    Lat: 33.256397, Lon: -116.235093
    Lat: 33.256245, Lon: -116.235201
    Lat: 33.256180, Lon: -116.235205
    Lat: 33.256050, Lon: -116.235153
    Lat: 33.255962, Lon: -116.235049
    Lat: 33.255862, Lon: -116.235003
    Lat: 33.255730, Lon: -116.234932
    Lat: 33.255665, Lon: -116.234933
    Lat: 33.255570, Lon: -116.234960
    Lat: 33.255528, Lon: -116.235016
    Lat: 33.255490, Lon: -116.235208
    Lat: 33.255487, Lon: -116.235346
    Lat: 33.25

    Lat: 33.269168, Lon: -116.237220
    Lat: 33.269173, Lon: -116.237288
    Lat: 33.269138, Lon: -116.237330
    Lat: 33.269151, Lon: -116.237358
    Lat: 33.269147, Lon: -116.237387
    Lat: 33.269115, Lon: -116.237415
    Lat: 33.269104, Lon: -116.237461
    Lat: 33.269066, Lon: -116.237520
    Lat: 33.269064, Lon: -116.237581
    Lat: 33.269089, Lon: -116.237627
    Lat: 33.269084, Lon: -116.237668
    Lat: 33.269033, Lon: -116.237698
    Lat: 33.269033, Lon: -116.237735
    Lat: 33.269080, Lon: -116.237778
    Lat: 33.269088, Lon: -116.237814
    Lat: 33.269042, Lon: -116.237884
    Lat: 33.269042, Lon: -116.237916
    Lat: 33.269089, Lon: -116.238026
    Lat: 33.269094, Lon: -116.238090
    Lat: 33.269040, Lon: -116.238216
    Lat: 33.269051, Lon: -116.238337
    Lat: 33.269038, Lon: -116.238501
    Lat: 33.269053, Lon: -116.238551
    Lat: 33.269132, Lon: -116.238592
    Lat: 33.269141, Lon: -116.238645
    Lat: 33.269109, Lon: -116.238718
    Lat: 33.268968, Lon: -116.238855
 

    Lat: 33.242647, Lon: -116.232527
    Lat: 33.242585, Lon: -116.232566
    Lat: 33.242514, Lon: -116.232563
    Lat: 33.242451, Lon: -116.232622
    Lat: 33.242370, Lon: -116.232659
    Lat: 33.242352, Lon: -116.232702
    Lat: 33.242322, Lon: -116.232727
    Lat: 33.242293, Lon: -116.232795
    Lat: 33.242274, Lon: -116.232888
    Lat: 33.242254, Lon: -116.232911
    Lat: 33.242165, Lon: -116.232929
    Lat: 33.242145, Lon: -116.232925
    Lat: 33.242113, Lon: -116.232884
    Lat: 33.242115, Lon: -116.232844
    Lat: 33.242134, Lon: -116.232821
    Lat: 33.242154, Lon: -116.232777
    Lat: 33.242156, Lon: -116.232756
    Lat: 33.242133, Lon: -116.232739
    Lat: 33.242106, Lon: -116.232753
    Lat: 33.242077, Lon: -116.232810
    Lat: 33.242041, Lon: -116.232874
    Lat: 33.241968, Lon: -116.232908
    Lat: 33.241939, Lon: -116.232955
    Lat: 33.241920, Lon: -116.232975
    Lat: 33.241874, Lon: -116.232983
    Lat: 33.241846, Lon: -116.232991
    Lat: 33.241842, Lon: -116.233013
 

    Lat: 33.256290, Lon: -116.238098
    Lat: 33.256209, Lon: -116.238098
    Lat: 33.256119, Lon: -116.238037
    Lat: 33.256043, Lon: -116.238012
    Lat: 33.255985, Lon: -116.238037
    Lat: 33.255946, Lon: -116.238023
    Lat: 33.255924, Lon: -116.237975
    Lat: 33.255868, Lon: -116.237948
    Lat: 33.255814, Lon: -116.237945
    Lat: 33.255765, Lon: -116.237940
    Lat: 33.255702, Lon: -116.237867
    Lat: 33.255646, Lon: -116.237894
    Lat: 33.255594, Lon: -116.237964
    Lat: 33.255603, Lon: -116.238045
    Lat: 33.255576, Lon: -116.238136
    Lat: 33.255493, Lon: -116.238246
    Lat: 33.255480, Lon: -116.238342
    Lat: 33.255410, Lon: -116.238380
    Lat: 33.255235, Lon: -116.238404
    Lat: 33.255157, Lon: -116.238361
    Lat: 33.255090, Lon: -116.238259
    Lat: 33.255009, Lon: -116.238243
    Lat: 33.254933, Lon: -116.238310
Name: n/a
  Highway: n/a
  Nodes:
    Lat: 33.256558, Lon: -116.237264
    Lat: 33.256508, Lon: -116.237324
    Lat: 33.256452, Lon: -116.237351
    

    Lat: 33.254310, Lon: -116.218633
    Lat: 33.254310, Lon: -116.218603
    Lat: 33.254330, Lon: -116.218509
    Lat: 33.254321, Lon: -116.218417
    Lat: 33.254338, Lon: -116.218318
    Lat: 33.254364, Lon: -116.218244
    Lat: 33.254358, Lon: -116.218185
    Lat: 33.254336, Lon: -116.218155
    Lat: 33.254297, Lon: -116.218140
    Lat: 33.254233, Lon: -116.218151
    Lat: 33.254056, Lon: -116.218232
    Lat: 33.253982, Lon: -116.218234
    Lat: 33.253959, Lon: -116.218185
    Lat: 33.253964, Lon: -116.218140
    Lat: 33.253996, Lon: -116.218098
    Lat: 33.254126, Lon: -116.217995
    Lat: 33.254159, Lon: -116.217929
    Lat: 33.254164, Lon: -116.217869
    Lat: 33.254124, Lon: -116.217715
    Lat: 33.254008, Lon: -116.217583
    Lat: 33.253927, Lon: -116.217506
    Lat: 33.253875, Lon: -116.217481
    Lat: 33.253801, Lon: -116.217496
    Lat: 33.253706, Lon: -116.217597
    Lat: 33.253655, Lon: -116.217601
    Lat: 33.253637, Lon: -116.217587
    Lat: 33.253591, Lon: -116.217535
 

In [4]:
result.ways

[]